In [1]:
# Cell 1: Install dependencies (run only once)
!pip install -q transformers torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.1 MB/s eta 0:00:00:00:0100:01


In [2]:
# Cell 2: Import, device set, model & processor load

import torch
from transformers import SeamlessM4Tv2ForSpeechToText, AutoProcessor
import torchaudio

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load processor and model
model_id = "facebook/seamless-m4t-v2-large"
processor = AutoProcessor.from_pretrained(model_id)
model = SeamlessM4Tv2ForSpeechToText.from_pretrained(model_id).to(device)

2025-07-28 11:19:18.784242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753701558.933353      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753701558.976222      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.24G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Instantiating a decoder SeamlessM4Tv2Attention without passing `layer_idx` is not recommended and will lead to errors during the forward call, if caching is used. Please make sure to provide a `layer_idx` when creating this class.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [3]:
# Cell 3: Define the translation function
def translate_speech_to_english(audio_file):
    # Load and resample
    speech_array, sampling_rate = torchaudio.load(audio_file)
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        speech_array = resampler(speech_array)
    speech_array = speech_array.squeeze().numpy()

    # Preprocess
    inputs = processor(audios=speech_array, sampling_rate=16000, return_tensors="pt")

    # Move to device
    for k in inputs:
        inputs[k] = inputs[k].to(device)

    # Generate translation directly
    translated_tokens = model.generate(**inputs, tgt_lang="eng")

    # Decode
    translated_text = processor.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text

In [6]:
# Cell 4: Provide audio file path and run inference
audio_path = "/kaggle/input/audio-files/14765312647 14765312640/cv-corpus-21.0-delta-2025-03-14/hi/clips/common_voice_hi_41938751.mp3"

translated_text = translate_speech_to_english(audio_path)
print(f"\nTranslated Text:\n{translated_text}")


Translated Text:
Kashmir: The world of fear and anger


In [7]:
import json

# Create a dictionary with the translation
output_data = {
    "audio_file": audio_path,
    "translated_text": translated_text
}

# Define output path
output_json_path = "translated_output.json"

# Save to JSON file
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)

print(f"\n✅ Translated text saved to {output_json_path}")
import shutil

# Move file to the output directory
shutil.move(output_json_path, f"/kaggle/working/{output_json_path}")

print(f"\n📁 You can find the JSON file in the 'working' directory of kaggle.")


✅ Translated text saved to translated_output.json

📁 You can find the JSON file in the 'working' directory of kaggle.


In [9]:
pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 55.6 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install indic-transliteration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
import speech_recognition as sr
from pydub import AudioSegment
from indic_transliteration.sanscript import transliterate, DEVANAGARI, ITRANS
import re

def convert_audio_to_wav(audio_path):
    if audio_path.lower().endswith('.wav'):
        return audio_path

    sound = AudioSegment.from_file(audio_path)
    wav_path = "converted_audio.wav"
    sound.export(wav_path, format="wav")
    return wav_path

def transcribe_audio_to_hindi(audio_path):
    recognizer = sr.Recognizer()
    try:
        wav_path = convert_audio_to_wav(audio_path)
        with sr.AudioFile(wav_path) as source:
            audio_data = recognizer.record(source)
        return recognizer.recognize_google(audio_data, language='hi-IN')
    except sr.UnknownValueError:
        return "Could not understand the audio."
    except sr.RequestError as e:
        return f"Could not request results; {e}"
    except Exception as e:
        return f"Error: {e}"

def clean_hinglish_text(raw_text):
    # Basic replacements for casual readability
    substitutions = {
        'aa': 'a',     # simplify long vowels
        'ii': 'i',
        'ee': 'i',
        'uu': 'u',
        'oo': 'u',
        'kh': 'kh',
        'gh': 'gh',
        'chh': 'chh',
        'jh': 'jh',
        'ph': 'f',
        'bh': 'bh',
        'sh': 'sh',
        'ṣ': 'sh',
        'ṛ': 'r',
        'ṭ': 't',
        'ḍ': 'd',
        'ñ': 'n',
        'ṅ': 'n',
    }

    # Lowercase and remove non-alpha characters
    text = raw_text.lower()
    for k, v in substitutions.items():
        text = text.replace(k, v)

    # Optional: remove duplicate vowels/consonants if needed
    text = re.sub(r'\s+', ' ', text)  # normalize spaces
    return text

def transliterate_to_casual_hinglish(hindi_text):
    try:
        raw = transliterate(hindi_text, DEVANAGARI, ITRANS)
        cleaned = clean_hinglish_text(raw)
        return cleaned
    except Exception as e:
        return f"Transliteration Error: {e}"

if __name__ == "__main__":
    audio_file = input("Enter path to your audio file: ")
    hindi_text = transcribe_audio_to_hindi(audio_file)
    print("\nHindi (Devanagari):", hindi_text)

    if "Error" not in hindi_text and "Could not" not in hindi_text:
        hinglish = transliterate_to_casual_hinglish(hindi_text)
        print("\nTranslated Text:", hinglish)
    else:
        print("\nNo valid transcription to transliterate.")

Enter path to your audio file:  /kaggle/input/audio-files/14765312647 14765312640/cv-corpus-21.0-delta-2025-03-14/hi/clips/common_voice_hi_41938750.mp3



Hindi (Devanagari): पाकिस्तानी राष्ट्रपति परवेज मुशर्रफ ने अपने पद से इस्तीफा दिया

Translated Text: pakistani rashtrapati paraveja musharrafa ne apane pada se istifa diya
